In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import glob

# Base directory path
base_dir = r"C:\Users\nacha\OneDrive\Desktop\school stuff\year 3 semester 1\visualization\vis proj\second try"
data_dir = os.path.join(base_dir, "data")

# Month mappings for processing
months = {
    "202409": "September 2024",
    "202412": "December 2024", 
    "202503": "March 2025",
    "202506": "June 2025"
}

def process_month_data(month_code):
    """Process all CSV files for a given month"""
    month_folder = os.path.join(data_dir, f"{month_code}-citibike-tripdata")
    
    # Get all CSV files in the month folder
    csv_files = glob.glob(os.path.join(month_folder, "*.csv"))
    
    print(f"Processing {month_code}: Found {len(csv_files)} files")
    
    # Read and combine all CSV files for this month
    all_data = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            all_data.append(df)
            print(f"  Loaded {file}: {len(df)} rows")
        except Exception as e:
            print(f"  Error loading {file}: {e}")
    
    if not all_data:
        print(f"No data found for {month_code}")
        return None
    
    # Combine all data
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"  Combined data: {len(combined_df)} total rows")
    
    # Convert datetime columns
    combined_df['started_at'] = pd.to_datetime(combined_df['started_at'])
    combined_df['ended_at'] = pd.to_datetime(combined_df['ended_at'])
    
    # Extract date for daily aggregation
    combined_df['start_date'] = combined_df['started_at'].dt.date
    combined_df['end_date'] = combined_df['ended_at'].dt.date
    
    # Remove rows with missing station information
    combined_df = combined_df.dropna(subset=['start_station_id', 'end_station_id', 
                                           'start_station_name', 'end_station_name',
                                           'start_lat', 'start_lng', 'end_lat', 'end_lng'])
    
    print(f"  After cleaning: {len(combined_df)} rows")
    
    return combined_df

def calculate_daily_station_balance(df):
    """Calculate daily net balance for each station"""
    
    # Create station master list with coordinates
    start_stations = df[['start_station_id', 'start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
    start_stations.columns = ['station_id', 'station_name', 'lat', 'lng']
    
    end_stations = df[['end_station_id', 'end_station_name', 'end_lat', 'end_lng']].drop_duplicates()
    end_stations.columns = ['station_id', 'station_name', 'lat', 'lng']
    
    # Combine and deduplicate stations
    all_stations = pd.concat([start_stations, end_stations]).drop_duplicates(subset=['station_id'])
    
    # Group by date and calculate departures (bikes leaving stations)
    departures = df.groupby(['start_date', 'start_station_id']).size().reset_index()
    departures.columns = ['date', 'station_id', 'departures']
    
    # Group by date and calculate arrivals (bikes arriving at stations)
    arrivals = df.groupby(['end_date', 'end_station_id']).size().reset_index()
    arrivals.columns = ['date', 'station_id', 'arrivals']
    
    # Get all unique dates
    all_dates = sorted(set(df['start_date'].unique()) | set(df['end_date'].unique()))
    
    daily_balances = []
    
    for date in all_dates:
        # Get departures and arrivals for this date
        date_departures = departures[departures['date'] == date]
        date_arrivals = arrivals[arrivals['date'] == date]
        
        # Merge with station master list
        date_balance = all_stations.copy()
        date_balance['date'] = date
        
        # Add departures
        date_balance = date_balance.merge(date_departures[['station_id', 'departures']], 
                                        on='station_id', how='left')
        date_balance['departures'] = date_balance['departures'].fillna(0)
        
        # Add arrivals
        date_balance = date_balance.merge(date_arrivals[['station_id', 'arrivals']], 
                                        on='station_id', how='left')
        date_balance['arrivals'] = date_balance['arrivals'].fillna(0)
        
        # Calculate net balance (positive = gained bikes, negative = lost bikes)
        date_balance['net_balance'] = date_balance['arrivals'] - date_balance['departures']
        
        # Only include stations that had activity
        date_balance = date_balance[(date_balance['departures'] > 0) | (date_balance['arrivals'] > 0)]
        
        daily_balances.append(date_balance)
    
    return pd.concat(daily_balances, ignore_index=True)

# Process each month
processed_data = {}

for month_code, month_name in months.items():
    print(f"\n=== Processing {month_name} ({month_code}) ===")
    
    # Load and process raw data
    raw_data = process_month_data(month_code)
    
    if raw_data is not None:
        # Calculate daily balances
        daily_balances = calculate_daily_station_balance(raw_data)
        
        # Save processed data
        output_file = os.path.join(base_dir, f"processed_{month_code}.csv")
        daily_balances.to_csv(output_file, index=False)
        
        processed_data[month_code] = daily_balances
        
        print(f"  Saved {len(daily_balances)} station-day records to {output_file}")
        print(f"  Date range: {daily_balances['date'].min()} to {daily_balances['date'].max()}")
        print(f"  Unique stations: {daily_balances['station_id'].nunique()}")
        print(f"  Average daily net balance: {daily_balances['net_balance'].mean():.2f}")
    else:
        print(f"  Skipped {month_name} - no data found")

print(f"\n=== Processing Complete ===")
print(f"Processed {len(processed_data)} months")

# Display sample data
if processed_data:
    sample_month = list(processed_data.keys())[0]
    print(f"\nSample data from {sample_month}:")
    print(processed_data[sample_month].head())
    print(f"\nData types:")
    print(processed_data[sample_month].dtypes)


=== Processing September 2024 (202409) ===
Processing 202409: Found 5 files


C:\Users\nacha\AppData\Local\Temp\ipykernel_4788\650649952.py:32: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


  Loaded C:\Users\nacha\OneDrive\Desktop\school stuff\year 3 semester 1\visualization\vis proj\second try\data\202409-citibike-tripdata\202409-citibike-tripdata_1.csv: 1000000 rows


C:\Users\nacha\AppData\Local\Temp\ipykernel_4788\650649952.py:32: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


  Loaded C:\Users\nacha\OneDrive\Desktop\school stuff\year 3 semester 1\visualization\vis proj\second try\data\202409-citibike-tripdata\202409-citibike-tripdata_2.csv: 1000000 rows


...

  Loaded C:\Users\nacha\OneDrive\Desktop\school stuff\year 3 semester 1\visualization\vis proj\second try\data\202506-citibike-tripdata\202506-citibike-tripdata_3.csv: 1000000 rows


C:\Users\nacha\AppData\Local\Temp\ipykernel_4788\650649952.py:32: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


  Loaded C:\Users\nacha\OneDrive\Desktop\school stuff\year 3 semester 1\visualization\vis proj\second try\data\202506-citibike-tripdata\202506-citibike-tripdata_4.csv: 1000000 rows


C:\Users\nacha\AppData\Local\Temp\ipykernel_4788\650649952.py:32: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


  Loaded C:\Users\nacha\OneDrive\Desktop\school stuff\year 3 semester 1\visualization\vis proj\second try\data\202506-citibike-tripdata\202506-citibike-tripdata_5.csv: 759345 rows
  Combined data: 4759345 total rows
  After cleaning: 4740706 rows
  Saved 93286 station-day records to C:\Users\nacha\OneDrive\Desktop\school stuff\year 3 semester 1\visualization\vis proj\second try\processed_202506.csv
  Date range: 2025-05-31 to 2025-06-30
  Unique stations: 4172
  Average daily net balance: 0.00

=== Processing Complete ===
Processed 4 months

Sample data from 202409:
  station_id                   station_name        lat        lng        date  \
0    5788.13          Lafayette St & E 8 St  40.730207 -73.991026  2024-08-31   
1    6593.15                6 Ave & W 45 St  40.756951 -73.982631  2024-08-31   
2    5847.01  Christopher St & Greenwich St  40.732916 -74.007114  2024-08-31   
3    5379.09              Berry St & N 8 St  40.719009 -73.958525  2024-08-31   
4    5288.12       Mur